In [26]:
import datetime
import rasterio
import utils


target_date = datetime.datetime(month=2, day=1, year=2022)
date_tolerance = 240 # plus and minus margin of target dates
max_cloud_percent = 30 # maximum amount of cloud permitted in a candidate scene

# the input raster, which defines our "area of interest"
AOI_raster = 'path/to/raster.tif'

# the directory where we store our whole satellite scenes
dl_directory = str('X:/storage/directory')

# 30m, 15m or 100m, for now I've only coded for 30m bands (B1, B2, B3, B4, B5, B6, B7)
# https://gisgeography.com/sentinel-2-bands-combinations/
target_bands = '30m'

In [27]:
start_date = target_date - datetime.timedelta(days=date_tolerance)
end_date = target_date + datetime.timedelta(days=date_tolerance)

if 'ls_login' not in globals():
    import getpass
    user = getpass.getpass(prompt = 'Enter EarthExplorer Username: ')      # Username for https://ers.cr.usgs.gov/
    password = getpass.getpass(prompt = 'Enter EarthExplorer Password: ')  # Password for https://ers.cr.usgs.gov/
    ls_login = [user, password]

from shapely.geometry import Polygon, MultiPolygon

dataset = rasterio.open(AOI_raster)
raster_extent = utils.get_spatial_extent(dataset, latlon=True)

# limit on the number of scenes that will be ordered, here set to 2 for testing
#order_limit = 2

In [28]:
import landsat_tools
ls_scenes = landsat_tools.ls_search_query(login=ls_login,
                            dataset='landsat_ot_c2_l2', # product type: see https://pypi.org/project/landsatxplore/
                            extent=raster_extent,
                            start_date=start_date,
                            end_date=end_date,
                            max_cloud_cover=max_cloud_percent
                           )

13 landsat orders wll be attempted


In [29]:
dl_keys = utils.check_dls(landsat=ls_scenes, dl_directory=dl_directory) #checks our orders against downloads in the folder 

for key, val in dl_keys.items():
    if len(val) > 0:
        if key == 'landsat':
            print('Downloading ' + key)
            ls_results = landsat_tools.ls_place_order(login=ls_login,
                                          scenes=val,
                                          dl_directory=dl_directory)
            output_directory = dl_directory + key
        if key == 'sentinel-2':
            print('Downloading ' + key)
            val = utils.s2_key_to_hash(val, s2_scenes)
            s2_results = sentinel_tools.s2_place_order(login=s2_login,
                                           scenes=val, 
                                           dl_directory=dl_directory)
            output_directory = dl_directory + key
        if key == 'planet':
            print('Downloading ' + key)
            val = utils.pl_key_to_hash(val, pl_scenes)
#            pl_results = [asyncio.create_task(planet_tools.pl_place_order(login=pl_API_key,
            pl_results = await planet_tools.pl_place_order(login=pl_API_key,
                                          scenes=val,
                                          dl_directory=dl_directory)
            output_directory = dl_directory + key

LC08_L2SP_003069_20220410_20220419_02_T1 is already downloaded
LC08_L2SP_003069_20220613_20220617_02_T1 is already downloaded
LC08_L2SP_003069_20220629_20220707_02_T1 is already downloaded
LC08_L2SP_003069_20220901_20220910_02_T1 is already downloaded
LC08_L2SP_003069_20220917_20220928_02_T1 is already downloaded
LC09_L2SP_003069_20220418_20220420_02_T1 is already downloaded
LC09_L2SP_003069_20220605_20220607_02_T1 is already downloaded
LC09_L2SP_003069_20220621_20220624_02_T1 is already downloaded
LC09_L2SP_003069_20220707_20220709_02_T1 is already downloaded
LC09_L2SP_003069_20220824_20220826_02_T1 is already downloaded
LC09_L2SP_003069_20220909_20220911_02_T1 is already downloaded
LC09_L2SP_003069_20220925_20220927_02_T1 is already downloaded
LC08_L2SP_003069_20210610_20210621_02_T1


100%|█████████████████████████████████████████████████████████████████████████████| 1.04G/1.04G [00:14<00:00, 79.1MB/s]


extracting D:/sat_dls/landsat\LC08_L2SP_003069_20210610_20210621_02_T1.tar


In [34]:
# the directory where we want to store our clipped satellite scenes
import os
ordername = AOI_raster.split("/")[-1].split(".")[0].split("_")[0]
output_directory = str(dl_directory + ordername + "_clipped")
target_ls_bands = '30m'

if not os.path.exists(output_directory):
    os.makedirs(output_directory)
top_directory_list = os.listdir(dl_directory)

In [35]:
ls_scn_dirs = utils.ls_directory_scns(dl_directory + 'landsat')

import landsat_tools
raster_extent = utils.get_spatial_extent(dataset, latlon=False)
pg = MultiPolygon([Polygon(raster_extent)])
pg_crs = dataset.crs
landsat_tools.ls_stack_n_crop(ls_scn_dirs, pg, pg_crs, target_ls_bands, output_directory)

writing layer 1
writing layer 2
writing layer 3
writing layer 4
writing layer 5
writing layer 6
writing layer 7
done writing scene 1, proceeding


C:\ProgramData\Anaconda3\envs\pyspat\lib\site-packages\rasterio\features.py:441: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  all_bounds = [bounds(shape, transform=~dataset.transform) for shape in shapes]
C:\ProgramData\Anaconda3\envs\pyspat\lib\site-packages\rasterio\features.py:284: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  for index, item in enumerate(shapes):


writing layer 1
writing layer 2
writing layer 3
writing layer 4
writing layer 5
writing layer 6
writing layer 7
done writing scene 2, proceeding
writing layer 1
writing layer 2
writing layer 3
writing layer 4
writing layer 5
writing layer 6
writing layer 7
done writing scene 3, proceeding
writing layer 1
writing layer 2
writing layer 3
writing layer 4
writing layer 5
writing layer 6
writing layer 7
done writing scene 4, proceeding
writing layer 1
writing layer 2
writing layer 3
writing layer 4
writing layer 5
writing layer 6
writing layer 7
done writing scene 5, proceeding
writing layer 1
writing layer 2
writing layer 3
writing layer 4
writing layer 5
writing layer 6
writing layer 7
done writing scene 6, proceeding
writing layer 1
writing layer 2
writing layer 3
writing layer 4
writing layer 5
writing layer 6
writing layer 7
done writing scene 7, proceeding
writing layer 1
writing layer 2
writing layer 3
writing layer 4
writing layer 5
writing layer 6
writing layer 7
done writing scene